# Tutorial data

In [ ]:
# import pandas as pd

# data_path = "data/train/day_0_0.001_train.csv"

# df = pd.read_csv(data_path, header=None)
# df.head()

In [ ]:
# example = df.loc[1, 0]
# example

In [ ]:
# features = example.split('\t')
# len(features)
# features

# Read s3 orc data

In [1]:
column_names = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

In [2]:
import metaspore as ms

# train_dataset_path = ROOT_DIR + '/data/train/day_0_0.001_train.csv'

file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 10
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

# # train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00000-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc'
# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
# train_dataset_path = [
#     's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/',
#     's3://mv-mtg-di-for-poc-datalab/2024/06/14/01/',
#     's3://mv-mtg-di-for-poc-datalab/2024/06/14/02/',
# ]


spark_confs = {
    'spark.eventLog.enabled':'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=2,
                                     server_count=2,
                                     log_level='WARN',
                                     spark_confs=spark_confs)


# train_dataset = ms.input.read_s3_csv(spark_session, 
#                                     train_dataset_path, 
#                                     format='orc',
#                                     shuffle=False,
#                                     delimiter='\t', 
#                                     multivalue_delimiter="\001", 
#                                     column_names=column_names,
#                                     multivalue_column_names=column_names[:-1])

# # train_dataset.printSchema()
# train_dataset.count()

/usr/local/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-22 05:54:04,561 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
from tqdm import tqdm
import pandas as pd
pd.set_option("display.max_columns", None)

columns = column_names[:-1]
feature_hash = {}
feature_value = {}

num_files_per_chunk = 10
num_chunks = (num_files + num_files_per_chunk - 1) // num_files_per_chunk

num_rows = 0

for i in tqdm(range(num_chunks), total=num_chunks):
    chunk_path = train_dataset_path[i*num_files_per_chunk:(i+1)*num_files_per_chunk]

    # Read the chunk
    train_dataset = ms.input.read_s3_csv(spark_session, 
                                     chunk_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])
    
    # Convert the chunk to a Pandas DataFrame
    df = train_dataset.toPandas()
    # df.head()
    num_rows += df.shape[0]

    # num_positive = df[df['label']=='1'].shape[0]
    # num_negative = df[df['label']=='0'].shape[0]
    # print(f"num_positive: {num_positive}, num_negative: {num_negative}, ratio: {num_positive/num_negative*100}")

    for i, row in df.iterrows(): # tqdm(df.iterrows(), total=df.shape[0]):
        for column in columns[:-1]:
            if column in feature_value:
                feature_value[column].append(row[column].shape[0])
            else:
                feature_value[column] = [row[column].shape[0]]
            
            for item in row[column]:
                if '\003' in item:
                    hash, weight = item.split('\003')
                else:
                    hash, weight = '', 0
                if column in feature_hash:
                    feature_hash[column].add(hash)
                else:
                    feature_hash[column] = set(hash)

print(f"Number of files: {num_files}, number of rows: {num_rows}")

  0%|          | 0/1 [00:00<?, ?it/s]

ignore shuffle


2024-07-22 05:56:38,650 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
100%|██████████| 1/1 [00:36<00:00, 36.84s/it]                                   

Number of files: 1, number of rows: 13800


In [5]:
df_feature = pd.DataFrame(feature_value)
df_feature.describe()

,_11001,_11002,_11003,_11004,_11007,_11008,_11021,_11022,_11023,_11024,_11041,_11042,_11043,_11044,_11045,_11046,_11061,_11062,_11063,_11064,_11065,_11066,_11081,_11082,_11083,_11084,_11085,_11086,_11601,_11602,_11603,_12001,_12002,_12003,_12004,_12005,_12006,_20001,_20002,_20003,_20101,_20102,_20201,_20202,_20203,_20204,_20205,_20206,_20207,_20208,_20209,_20210,_30001,_30002,_30003,_30004,_30005,_30006,_30201,_30202,_30203,_30204,_30205,_30206,_30207,_40001,_40002,_40003,_40004,_40005,_40201,_40202,_40203,_40204,_40205,_40206,_40207,_40208,_40209,_40210,_40211,_40212,_40213,_40214,_40215,_40231,_40301,_40302,_40303,_40304,_40305,_40306,_40307,_40321,_40322,_40323,_40324,_50801,_50802,_50805,_50806,_50807,_50810,_51001,_51002,_51003,_51004,_51005,_51006,_51011,_51012,_51013,_51014,_51021,_51022,_51023,_51024,_51025,_51026,_51031,_51032,_51033,_51034,_51035,_51036,_51041,_51042,_51043,_51044,_51045,_51046,_52001,_52002,_52003,_52004,_52005,_61101,_61102,_61103,_70001,_70002,_70003,_70004,_70005,_70006,_70007,_70008,_70009,_70010,_70011,_70031,_70032,_70033,_70034,_70035,_70036,_70037,_70038,_70039,_70040,_70041,_70301,_70302,_70303,_70304,_70305,_70306,_70601,_70602,_70603,_70604,_70605,_70606,_70901,_70902,_70903,_70904,_70905,_70906,_72401,_72402,_80001,_80002,_80003,_80004,_80005,_80006,_80007,_80008,_80009,_80010,_80011,_80012,_80013,_80014,_80015,_80016,_80017,_80018,_80019,_80020,_80021,_80022,_80023,_80024,_80025,_80026,_80027,_80028,_80029,_80030,_80031,_80032,_80033,_80034,_80035,_80036,_80037,_80038,_80039,_80040,_80041,_80042,_80043,_80044,_80045,_80046,_80047,_80048,_80049,_80050,_80051,_80052,_80053,_80054,_80055,_80056,_80057,_80058,_80059,_80060,_80061,_80062,_80063,_80064,_80065,_80066,_80067,_80068,_80069,_80070,_80071,_80072,_80073,_80074,_80075,_80076,_80077,_80078,_80079,_80080,_80081,_80082,_80083,_80084,_80085,_80086,_80087,_80088,_80089,_80090,_80091,_80092,_80093,_80094,_80095,_80096,_80097,_80098,_80099,_80100,_80101,_80102,_80103,_80104,_80105,_80106,_80107,_80108,_80109,_80110,_80111,_80112,_80113,_80114,_80115,_80125,_80126,_80127,_80128,_80129,_80130,_80131,_80132,_80133,_80134,_80135,_80136,_80137,_80138,_80139,_80140,_80141,_80142,_80143,_80144,_80145,_80146,_80147,_80148,_80149,_80150
count,13800.000000,13800.000000,13800.0,13800.0,13800.000000,13800.0,13800.000000,13800.000000,13800.0,13800.0,13800.000000,13800.000000,13800.000000,13800.0,13800.0,13800.0,13800.000000,13800.000000,13800.000000,13800.0,13800.0,13800.0,13800.000000,13800.000000,13800.000000,13800.0,13800.0,13800.0,13800.000000,13800.000000,13800.000000,13800.000000,13800.000000,13800.000000,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.000000,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.000000,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.0,13800.000000,13800.000000,13800.0,13800.0,13800.000000,13800.000000,13800.0,13800.000000,13800.000000

In [6]:
feature_hash_count = dict([(k, [len(v)]) for k, v in feature_hash.items()])
df_hash = pd.DataFrame(feature_hash_count)
df_hash

,_11001,_11002,_11003,_11004,_11007,_11008,_11021,_11022,_11023,_11024,_11041,_11042,_11043,_11044,_11045,_11046,_11061,_11062,_11063,_11064,_11065,_11066,_11081,_11082,_11083,_11084,_11085,_11086,_11601,_11602,_11603,_12001,_12002,_12003,_12004,_12005,_12006,_20001,_20002,_20003,_20101,_20102,_20201,_20202,_20203,_20204,_20205,_20206,_20207,_20208,_20209,_20210,_30001,_30002,_30003,_30004,_30005,_30006,_30201,_30202,_30203,_30204,_30205,_30206,_30207,_40001,_40002,_40003,_40004,_40005,_40201,_40202,_40203,_40204,_40205,_40206,_40207,_40208,_40209,_40210,_40211,_40212,_40213,_40214,_40215,_40231,_40301,_40302,_40303,_40304,_40305,_40306,_40307,_40321,_40322,_40323,_40324,_50801,_50802,_50805,_50806,_50807,_50810,_51001,_51002,_51003,_51004,_51005,_51006,_51011,_51012,_51013,_51014,_51021,_51022,_51023,_51024,_51025,_51026,_51031,_51032,_51033,_51034,_51035,_51036,_51041,_51042,_51043,_51044,_51045,_51046,_52001,_52002,_52003,_52004,_52005,_61101,_61102,_61103,_70001,_70002,_70003,_70004,_70005,_70006,_70007,_70008,_70009,_70010,_70011,_70031,_70032,_70033,_70034,_70035,_70036,_70037,_70038,_70039,_70040,_70041,_70301,_70302,_70303,_70304,_70305,_70306,_70601,_70602,_70603,_70604,_70605,_70606,_70901,_70902,_70903,_70904,_70905,_70906,_72401,_72402,_80001,_80002,_80003,_80004,_80005,_80006,_80007,_80008,_80009,_80010,_80011,_80012,_80013,_80014,_80015,_80016,_80017,_80018,_80019,_80020,_80021,_80022,_80023,_80024,_80025,_80026,_80027,_80028,_80029,_80030,_80031,_80032,_80033,_80034,_80035,_80036,_80037,_80038,_80039,_80040,_80041,_80042,_80043,_80044,_80045,_80046,_80047,_80048,_80049,_80050,_80051,_80052,_80053,_80054,_80055,_80056,_80057,_80058,_80059,_80060,_80061,_80062,_80063,_80064,_80065,_80066,_80067,_80068,_80069,_80070,_80071,_80072,_80073,_80074,_80075,_80076,_80077,_80078,_80079,_80080,_80081,_80082,_80083,_80084,_80085,_80086,_80087,_80088,_80089,_80090,_80091,_80092,_80093,_80094,_80095,_80096,_80097,_80098,_80099,_80100,_80101,_80102,_80103,_80104,_80105,_80106,_80107,_80108,_80109,_80110,_80111,_80112,_80113,_80114,_80115,_80125,_80126,_80127,_80128,_80129,_80130,_80131,_80132,_80133,_80134,_80135,_80136,_80137,_80138,_80139,_80140,_80141,_80142,_80143,_80144,_80145,_80146,_80147,_80148,_80149,_80150
0,7958,7713,10,12,8349,10,5476,6648,2,2,5023,6740,7869,11,9,10,4607,8359,8467,2,11,10,16721,22814,28941,11,10,10,39870,22090,9501,26740,26055,25458,10,8,9,186,3722,77,16,14,164,1857,107,13,18,15,71,16,71,16,10,15,4168,5837,5631,966,4063,1873,2038,892,18,55,110,507,2589,3344,23,8,14,948,12,1818,10,20,20,11,26,28,2828,4420,1347,583,698,14,2438,1086,1116,487,16,55,114,44,16,25,1262,9,9,8,9,11,10,361147,2,312183,10,386132,11,85491,2,151585,2,27140,2,62589,2,102318,2,11437,2,28381,2,28767,2,91694,2,157682,2,225357,2,214345,2,179350,2,160469,2,2,2,11041,11919,3591,6705,7494,7093,3062,5761,12530,12105,12541,11276,12109,4557,7615,8313,7521,3967,6790,12792,12278,12797,5421,7191,77,308,516,9,7965,9439,245,1001,1518,2,8688,10055,417,1545,2188,2,2,2,3366,9009,108,208,2642,2893,1520,3679,33,35,3099,5616,108,90,72,65,69,67,56,107,74,68,62,61,54,46,47,36,34,33,39,35,32,109,70,37,39,4620,2697,108,57,3404,1225,446,3553,33,33,1276,5074,104,79,58,58,55,44,44,97,60,51,43,43,36,34,25,20,16,20,19,21,16,89,78,53,39,12008,109,3866,1282,4211,36,31,32,30,32,32,25,9065,109,6244,174,3766,51,50,48,52,50,49,44,16,16,16,15,15,16,18,11,16,10,16,17,15,9,18,15,17,16,17,17,15,15,17,17,16,9,171060,10613,10214,56140,2765,8970,4333,4244,37221,2123,105417,12697,8973,11405,56657,26197,8312


In [7]:
hashes = set()

for k, v in feature_hash.items():
    hashes |= v
print(f"Number of total hashes: {len(hashes)}")

Number of total hashes: 1948315


In [8]:
total_hash = df_hash.sum(axis=1).values[0]
print(f"Number of total hashes: {total_hash}")

Number of total hashes: 3826716


# Use pyspark to analyze

In [ ]:
columns = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

In [ ]:
import metaspore as ms
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode, sum as spark_sum, countDistinct
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType

# Read column names from file
columns = []
with open('./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

# Define Spark configuration and session
file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 1
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

spark_confs = {
    'spark.eventLog.enabled': 'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=3,
                                     server_count=3,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

# Load the dataset
train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=columns,
                                     multivalue_column_names=columns[:-1])

print(f"Number of orcs: {num_files}, total number of rows: {train_dataset.count()}")

# Define UDF to process rows
def process_row(column_values):
    feature_hashes = set()
    for value in column_values:
        if '\003' in value:
            hash_val, weight = value.split('\003')
        else:
            hash_val, weight = '', 0
        feature_hashes.add(hash_val)

    feature_values = len(column_values)
    return (feature_values, list(feature_hashes))

# Define schema for UDF output
schema = StructType([
    StructField("feature_values", IntegerType(), True),
    StructField("hashes", ArrayType(StringType()), True)
])

# Register the UDF
process_row_udf = udf(process_row, schema)

# Apply UDF on each column and process the dataset
for column in columns[:-1]:
    train_dataset = train_dataset.withColumn(f'{column}_processed', process_row_udf(col(column)))

train_dataset.show()


In [ ]:
# Extract the processed columns
result_columns = [f'{column}_processed' for column in columns[:-1]]
processed_df = train_dataset.select(*result_columns)
processed_df.show()

In [ ]:
# Select the desired row and column
row_value = train_dataset.select('_11002_processed').collect()[4][0]

# Print the value
print(row_value)

In [ ]:
# 1. Obtain the percentiles of feature_values using approxQuantile
from tqdm import tqdm
percentiles = [0.25, 0.5, 0.75, 1.0]

for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    feature_values_df = processed_df.select(col(f"{col_name}.feature_values").cast("double").alias("feature_values"))
    
    # Print schema to verify cast
    # feature_values_df.printSchema()
    
    # Show data to verify cast
    # feature_values_df.show()
    
    # Verify the data type of the column
    # data_type = feature_values_df.schema["feature_values"].dataType
    # print(f"Data type of feature_values in {col_name}: {data_type}")
    
    # # Check for null values or any anomalies in the column
    # null_count = feature_values_df.filter(col("feature_values").isNull()).count()
    # print(f"Number of null values in feature_values of {col_name}: {null_count}")
    
    # Obtain approximate percentiles
    try:
        percentile_values = feature_values_df.approxQuantile("feature_values", percentiles, 0.01)
        # print(f"Percentiles for {col_name}: {percentile_values}")
    except Exception as e:
        print(f"Error occurred while calculating percentiles for {col_name}: {e}")

# 2. Obtain the total number of different hash values for each column and across all columns
distinct_hash_counts = {}
all_hashes_df = []

print(f"Starting counting hashes")
for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    hashes_df = processed_df.select(explode(col(f"{col_name}.hashes")).alias("hashes"))
    distinct_count = hashes_df.select(countDistinct("hashes")).first()[0]
    distinct_hash_counts[col_name] = distinct_count
    all_hashes_df.append(hashes_df)

union_hashes_df = all_hashes_df[0]
for df in all_hashes_df[1:]:
    union_hashes_df = union_hashes_df.union(df)

total_distinct_hashes = union_hashes_df.select(countDistinct("hashes")).first()[0]

print(f"Distinct hash counts per column: {distinct_hash_counts}")
print(f"Total distinct hashes across all columns: {total_distinct_hashes}")

# Stop the Spark session
# spark.stop()